In [ ]:
import numpy as np
import pandas as pd
import os


In [1]:
# =============================================================================
# STEP 0. UPLOAD DATA BASE AND ROUTES.
# =============================================================================
BaseAumentada = pd.read_csv(r"/content/BaseLD.csv",header=0,sep=",")

pertenencia = np.zeros((27751, 1))  # matrix(0, nrow=27751, ncol=1)

In [2]:
# =============================================================================
# STEP 2. ALGORITHM.
# =============================================================================

m = BaseAumentada.shape[0]  # nrow(BaseAumentada)
n = BaseAumentada.shape[1]  # ncol(BaseAumentada)

print(BaseAumentada.columns)  # colnames(BaseAumentada)

neuronas = np.zeros((3, n))   # matrix(0, nrow=3, ncol=n)
promedio = np.zeros((1, n))   # matrix(0, nrow=1, ncol=n)
minimo   = np.zeros((1, n))
maximo   = np.zeros((1, n))

# Initialize the locations.

# for(i in 1:n) { minimo[1,i]=min(BaseAumentada[,i]) }
for i in range(n):
    minimo[0, i] = BaseAumentada.iloc[:, i].min()

# for(i in 1:n) { maximo[1,i]=max(BaseAumentada[,i]) }
for i in range(n):
    maximo[0, i] = BaseAumentada.iloc[:, i].max()

# for(i in 1:n) { promedio[1,i]=mean(BaseAumentada[,i]) }
for i in range(n):
    promedio[0, i] = BaseAumentada.iloc[:, i].mean()

neuronas[0, :] = minimo[0, :]
neuronas[1, :] = promedio[0, :]
neuronas[2, :] = maximo[0, :]

Index(['SegmentR', 'PEPR', 'AgeR', 'TimeRelationR', 'IncomeSourceR',
       'ActivityR', 'NationalityR', 'CommonAddR', 'ProductTR', 'SubproductR',
       'OperationTR', 'MonetaryInstR', 'DivisaR', 'ChannelR', 'OperabilitySR',
       'OGoverR', 'SARR', 'VMR', 'VOR', 'SumSeR', 'SumReR', 'RSR', 'AAR',
       'SendingNumR', 'ReceivingNumR', 'StateOpeR', 'StateResR', 'MunOpeR',
       'MunResR', 'CountryR'],
      dtype='object')


In [3]:
# Synaptic Potential

distancia = np.zeros((1, 3))  # matrix(0, nrow=1, ncol=3)
base = BaseAumentada.to_numpy()  # as.matrix(BaseAumentada)

# Cluster
cluster = np.zeros((1, 3))

for k in range(1, 4):  # for (k in 1:3)
    cluster = np.zeros((1, 3))

    for i in range(m):  # for(i in 1:m)

        for j in range(3):  # for(j in 1:3)
            # distancia[,j] <- norm(as.matrix(base[i,]-neuronas[j,]),"f")
            distancia[0, j] = np.linalg.norm(base[i, :] - neuronas[j, :], ord=2)

        un = 0.5 * (1 - (i + 1) / m)  # 0.5*(1 - i/m) with R's 1-based i

        min_dist = distancia.min()  # min(distancia[,1],distancia[,2],distancia[,3])

        # If there are ties, R code can update more than one neuron.
        if distancia[0, 0] == min_dist:
            neuronas[0, :] = neuronas[0, :] + un * (base[i, :] - neuronas[0, :])
            cluster[0, 0] += 1
            pertenencia[i, 0] = 1

        if distancia[0, 1] == min_dist:
            neuronas[1, :] = neuronas[1, :] + un * (base[i, :] - neuronas[1, :])
            cluster[0, 1] += 1
            pertenencia[i, 0] = 2

        if distancia[0, 2] == min_dist:
            neuronas[2, :] = neuronas[2, :] + un * (base[i, :] - neuronas[2, :])
            cluster[0, 2] += 1
            pertenencia[i, 0] = 3

In [4]:
# =============================================================================
# STEP 3. RESULTS.
# =============================================================================

# Print Neurons
print(neuronas[0, :])
print(neuronas[1, :])
print(neuronas[2, :])

nombre_neurona = ["Neurona1", "Neurona2", "Neurona3"]

# neuronas.p <- cbind(nombre.neurona, neuronas)
neuronas_p = pd.DataFrame(neuronas)
neuronas_p.insert(0, "nombre.neurona", nombre_neurona)

[44.28350274 10.03961845 45.67423291 35.99308255 44.06956404 35.1751072
 10.04053227  9.89715431 24.94328816 32.42291861  4.07016813 51.56672818
 34.41254702 31.2283498  46.72342009 10.10447752 14.75745519 31.35407732
 30.4970414  15.16711827 14.45898116 14.68019544 20.73702476 12.64252361
 11.62585632 24.84288141 28.73491878 15.05926329  4.92397647  9.98327113]
[19.97709498  9.88600297 33.08777494 19.98001788 25.04179832 19.93348306
  9.81472769 10.05646085 24.88781428  7.45653481  4.05715339 19.89011741
 19.94476373 19.90561318 11.09108851 10.13686121 15.424565   29.72075566
 31.61351013 14.70140954 13.00137195 14.97184449 18.83591272 13.0962767
 12.1897412  24.89266947 15.12073616 15.16060833  4.8851251   9.98254942]
[74.73982987 83.83934134 93.54241684 97.97021315 97.64789092 77.1962116
 41.59374833 10.44097012 33.71535972 87.63528762 53.15892944 81.30594448
 71.554964   64.22454404 97.99229966 33.13477481 25.27661112 30.37608063
 35.76905541 18.48873057 25.09402746 35.17002409 33.

In [5]:
# Print means (your R code divides by 30 explicitly)
sump1 = neuronas[0, :].sum() / 30
sump2 = neuronas[1, :].sum() / 30
sump3 = neuronas[2, :].sum() / 30

suma_promedio = pd.DataFrame({
    "nombre.neurona": nombre_neurona,
    "suma.promedio": [sump1, sump2, sump3]
})

suma_promedio

,nombre.neurona,suma.promedio
0,Neurona1,24.003643
1,Neurona2,16.624813
2,Neurona3,53.732255


In [7]:
cluster_df = pd.DataFrame({
    "nombre.neurona": nombre_neurona,
    "total.elementos": cluster.flatten().astype(int)
})

cluster_df

,nombre.neurona,total.elementos
0,Neurona1,199
1,Neurona2,13962
2,Neurona3,13590


In [8]:
# Print Ownership
pertenencia_df = pd.DataFrame(pertenencia, columns=["pertenencia"])


# base.actualizada <- cbind(pertenencia, BaseAumentada)
base_actualizada = pd.concat([pertenencia_df, BaseAumentada.reset_index(drop=True)], axis=1)
print(base_actualizada)

       pertenencia  SegmentR  PEPR  AgeR  TimeRelationR  IncomeSourceR  \
0              3.0        80   100    96            100            100   
1              3.0        96    96    97             98             99   
2              3.0        81    98    99             99             99   
3              3.0        96    99    99            100             96   
4              3.0        82    99    98             98             97   
...            ...       ...   ...   ...            ...            ...   
27746          2.0        19    12    32             19             27   
27747          2.0        20     9    32             18             25   
27748          2.0        22    10    34             21             26   
27749          2.0        20    12    33             20             24   
27750          2.0        20    11    34             21             26   

       ActivityR  NationalityR  CommonAddR  ProductTR  ...  SumReR  RSR  AAR  \
0             97            96 